In [ ]:
import gymnasium as gym
from gymnasium.spaces import Discrete, Box, Sequence, Dict
import numpy as np
from utils import methods
import statistics

from utils.rl_environments.env1 import Env1

In [ ]:
from stable_baselines3.common.env_checker import check_env
c = Env1()
# check_env(c, warn=True)
# experiment with the environment
obs, info = c.reset()
print(info['hidden'])
done = False 
while not done:
    _, reward, done, _, info = c.step(np.random.choice([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))
    print(info['state'])
    print(reward)

{'alpha': 6, 'beta': 3.0, 'interval': array([40.7617508 , 15.54982947, 20.68427661, 10.02474855, 11.91237877,
        4.97582587, 33.05793731, 18.09322345, 22.88402456, 26.05541825,
       13.55167543, 19.65148996, 22.22419067, 11.51809691, 12.11013878,
       12.56621774]), 'cum_sum_intervals': array([ 40.7617508 ,  56.31158027,  76.99585688,  87.02060543,
        98.9329842 , 103.90881007, 136.96674737, 155.05997083,
       177.94399538, 203.99941363, 217.55108905, 237.20257901,
       259.42676968, 270.94486659, 283.05500537, 295.62122311])}
{'n': 3, 'N': 13, 'h': 0.13, 'c': 21, 'travel_time': 200.87295649690216, 'cur_time': 77.99585688114811, 'mean_n': 25.665285627049368, 'std_n': 13.32359119656653, 'alpha_hat': 3.7106466987014888, 'beta_hat': 6.916661086605397, 'u_star_hat': 283.26830081, 'last_update': 76.99585688114811}
0
{'n': 3, 'N': 13, 'h': 0.13, 'c': 21, 'travel_time': 200.87295649690216, 'cur_time': 78.99585688114811, 'mean_n': 25.665285627049368, 'std_n': 13.3235911965665

In [3]:
DEFAULT_CONFIG = {
    'alpha_range': range(1, 8),
    'beta_range': [round(i * 0.5, 1) for i in range(2, 9)],
    'h_range': [round(i * 0.01, 2) for i in range(6, 61)],
    'c_range': range(20, 30),
    'total': range(10, 40), 
}

CONSTANT_CONFIG = {
    'alpha_range': [1],
    'beta_range': [1],
    'h_range': [0.15],
    'c_range': [25],
    'total': range(10, 40), 
}


In [ ]:
import stable_baselines3
from stable_baselines3 import PPO, DQN
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import SubprocVecEnv



env = Env1(config=CONSTANT_CONFIG)

model = DQN("MlpPolicy", env, verbose=1, device='cpu', exploration_final_eps=0.1)


# Start training
model.learn(total_timesteps=20000)

model.save("dqn")

model = DQN.load("dqn")


state, _ = env.reset()
done = False
total_reward = 0
model.predict(state)

while not done:
    action, _ = model.predict(state)  
    state, reward, done, _, _ = env.step(action)
    total_reward += reward
    env.render()

print("Total reward:", total_reward)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 2.25     |
|    ep_rew_mean      | -12.5    |
|    exploration_rate | 0.996    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 233      |
|    time_elapsed     | 0        |
|    total_timesteps  | 9        |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 1.62     |
|    ep_rew_mean      | -15.7    |
|    exploration_rate | 0.994    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 232      |
|    time_elapsed     | 0        |
|    total_timesteps  | 13       |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 1.83     |
|    ep_rew_mean      | -14.7  

In [8]:
def get_realized_data(config):
    alpha = np.random.choice(config['alpha_range'])
    beta = np.random.choice(config['beta_range'])
    h = np.random.choice(config['h_range'])
    c = np.random.choice(config['c_range'])
    total = np.random.choice(config['total'])
    intervals = np.random.gamma(shape=alpha, scale=beta, size=total)
    travel_time = sum(intervals[4:]) - np.random.exponential(scale=beta)
    travel_time = max(beta * 2, travel_time)

    return alpha, beta, h, c, total, intervals, travel_time

import pandas as pd

df = pd.DataFrame(columns=['h', 'c', 'travel_time', 'total', 'intervals'])

for i in range(10000):
    alpha, beta, h, c, total, intervals, travel_time = get_realized_data(CONSTANT_CONFIG)
    row = {'h': h, 'c': c, 'travel_time': travel_time, 'total': total, 'intervals': intervals}
    df.loc[i] = row



In [ ]:
from stable_baselines3 import DQN
model = DQN.load("dqn")

env = Env1(config=CONSTANT_CONFIG)
rewards = {}
print(env._get_info())
for i in range(len(df)):
    row = df.iloc[i]
    state, _ = env.reset(row=row)
    done = False
    total_reward = 0
    while not done:
        action, _ = model.predict(state)  
        state, reward, done, _, info = env.step(action)
        total_reward += reward
    rewards[i] = total_reward
    # print(f"Episode {i} reward: {total_reward} | h = {info['state']['h']}, c = {info['state']['c']}")
    print(f"Episode {i} reward: {total_reward} | cur_time = {info['state']['cur_time']}")

df['rewards'] = rewards
print(f"Average reward: {df['rewards'].mean()}")
df.head()

{'hidden': {'alpha': -1, 'beta': -1, 'interval': -1, 'cum_sum_intervals': -1}, 'state': {'n': -1, 'N': -1, 'h': -1, 'c': -1, 'travel_time': -1, 'cur_time': -1, 'mean_n': -1, 'std_n': -1, 'alpha_hat': -1, 'beta_hat': -1, 'u_star_hat': -1, 'last_update': -1}}
Episode 0 reward: -25 | cur_time = 25.924255320531962
Episode 1 reward: -25 | cur_time = 8.509623087274282
Episode 2 reward: -25 | cur_time = 22.55449048999091
Episode 3 reward: -25 | cur_time = 9.480217378100374
Episode 4 reward: -25 | cur_time = 15.981370597729367
Episode 5 reward: -25 | cur_time = 9.10372487745279
Episode 6 reward: -25 | cur_time = 32.13487381283707
Episode 7 reward: -25 | cur_time = 12.64641096755581
Episode 8 reward: -25 | cur_time = 35.057124535109104
Episode 9 reward: -25 | cur_time = 22.58763309343142
Episode 10 reward: -25 | cur_time = 16.61619395980834
Episode 11 reward: -25 | cur_time = 20.18989663719226
Episode 12 reward: -25 | cur_time = 7.8213693249452
Episode 13 reward: -25 | cur_time = 7.716498602359

,h,c,travel_time,total,intervals,rewards
0,0.15,25,22.510459,29,"[0.31734480317393426, 0.2866535806167686, 0.32...",-25.0
1,0.15,25,9.352392,14,"[2.22525234303231, 1.9072373119581287, 1.37713...",-25.0
2,0.15,25,18.208550,23,"[0.7881487488214416, 0.5417148067904182, 1.224...",-25.0
3,0.15,25,30.566343,29,"[0.2604473894384296, 1.3913438126320932, 0.828...",-25.0
4,0.15,25,9.605006,15,"[0.09185161442180285, 1.741258167259322, 1.148...",-25.0
